In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm_notebook as tqdm
import os

import matplotlib.pyplot as plt
plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (12, 8)


<Figure size 432x288 with 0 Axes>

In [2]:
base_path = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data'

ignore = []

# ignore = ["Patient_0225", "Patient_0289", "Patient_0290", "Patient_0291", "Patient_0292", "Patient_0294", 
#           "Patient_0295", "Patient_0296", "Patient_0297", "Patient_0298", "Patient_0299", "Patient_0300", 
#           "Patient_0301", "Patient_0302", "Patient_0303", "Patient_0316", "Patient_0317", "Patient_0319", 
#           "Patient_0320", "Patient_0322", "Patient_0323", "Patient_0325", "Patient_0326", "Patient_0327", 
#           "Patient_0328", "Patient_0329", "Patient_0330", "Patient_0331", "Patient_0335", "Patient_0336", 
#           "Patient_0337", "Patient_0339", "Patient_0340", "Patient_0341", "Patient_0342", "Patient_0343",
#           "Patient_0429"]

t1_files = sorted([f for f in glob('{}/*.npy'.format(base_path)) if 'T2' not in f])
t1_files_filt = []
for t1f in t1_files:
    t2_fname = t1f.split('/')[-1].replace('.npy', '_T2.npy')
    case = t2_fname.replace('_T2.npy', '')
    t2_path = '{}/{}'.format(base_path, t2_fname)
    if os.path.isfile(t2_path) and not case in ignore:
        t1_files_filt.append(t1f)
t1_files = t1_files_filt
print(t1_files)

['/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0130.npy', '/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0148.npy', '/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0149.npy', '/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0151.npy', '/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0170.npy', '/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0225.npy', '/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0429.npy']


In [5]:
for fpath in sorted(glob('{}/*_full.npy'.format(base_path))):
    os.rename(fpath, fpath.replace('_full', ''))

In [4]:
for t1_file in tqdm(t1_files, total=len(t1_files)):
    try:
        t1_data = np.load(t1_file, mmap_mode='r+')
        t1_pre, t1_low, t1_full = t1_data.transpose(2, 0, 1, 3, 4)
        fpath_t2 = t1_file.replace('.npy', '_T2.npy')
        t2_data = np.load(fpath_t2)
        t1_new_file = t1_file.replace('.npy', '_full.npy')
        data_new = np.array([t1_pre, t1_low, t1_full, t2_data]).transpose(1, 2, 0, 3, 4)
        np.save(t1_new_file, data_new)
        
        print('Saving new file to {}...'.format(t1_new_file))
        os.remove(fpath_t2)
        os.remove(t1_file)
    except Exception as exc:
        print('****Error in', t1_file)
        print('Error', exc)
        print('t1 shape', t1_data.shape)
        print('t2 shape', t2_data.shape, '****')

Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0130_full.npy...
Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0148_full.npy...
Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0149_full.npy...
Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0151_full.npy...
Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0170_full.npy...
Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0225_full.npy...
Saving new file to /home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0429_full.npy...



In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0289.npy', mmap_mode='r+')